In [185]:
from pyspark.sql import SparkSession, dataframe, Row
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType, FloatType
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.functions import col,trim,ltrim,rtrim,when,regexp_replace,concat_ws, lit, sha2, upper

import os
import re 

In [186]:
#sessão do spark com o Hive
#spark = SparkSession.builder.master("local[*]")\
#    .enableHiveSupport()\
#    .getOrCreate()

# CLIENTES

In [196]:
#Iniciando o tratamento dos dados
df_clientes = spark.sql("select * from desafio_curso.tbl_clientes")

In [197]:
#Removendo colunas desnecessárias
df_clientes = df_clientes.drop("business_unit","customer_type","phone","regional_sales_mgr")

In [198]:
#Convertendo os tipos de dados
df_clientes = df_clientes.withColumn("address_number",col("address_number").cast(IntegerType()))\
        .withColumn("customerkey",col("customerkey").cast(IntegerType()))\
        .withColumn("division",col("division").cast(IntegerType()))\
        .withColumn("region_code",col("region_code").cast(IntegerType()))

In [199]:
#Tratando as colunas vazias
df_clientes = df_clientes.withColumn('line_of_business', regexp_replace('line_of_business', '^\s+$', 'Não Informado'))

In [201]:
df_clientes = df_clientes.select("address_number"
        ,"business_family"
        ,"customerkey"
        ,"division"
        ,"line_of_business"
        ,"region_code"
        ,"search_type"
        ,"dt_foto"
        ,upper(col("customer")).alias("customer")
    )

In [202]:
#Removendo tuplas duplicadas
df_clientes = df_clientes.distinct()

In [204]:
df_clientes.printSchema()

root
 |-- address_number: integer (nullable = true)
 |-- business_family: string (nullable = true)
 |-- customerkey: integer (nullable = true)
 |-- division: integer (nullable = true)
 |-- line_of_business: string (nullable = true)
 |-- region_code: integer (nullable = true)
 |-- search_type: string (nullable = true)
 |-- dt_foto: string (nullable = true)
 |-- customer: string (nullable = true)



In [206]:
df_clientes.show()

+--------------+---------------+-----------+--------+----------------+-----------+-----------+--------+--------------------+
|address_number|business_family|customerkey|division|line_of_business|region_code|search_type| dt_foto|            customer|
+--------------+---------------+-----------+--------+----------------+-----------+-----------+--------+--------------------+
|      10017828|             R1|   10017828|       2|   Não Informado|          1|          C|20230630|       LEE MEGASTORE|
|      10018352|             R3|   10018352|       1|   Não Informado|          5|          C|20230630|       JOHNSON STORE|
|      10020662|             R3|   10020662|       1|              M1|          5|          C|20230630|       NEMATRON SHOP|
|      10001784|             R2|   10001784|       1|   Não Informado|          5|          C|20230630|         ACACIA SHOP|
|      10018867|             R3|   10018867|       2|   Não Informado|          4|          C|20230630|    KARI SUPERMARKET|


In [207]:
df_clientes.createOrReplaceTempView('tb_clientes')

# DIVISÃO

In [208]:
df_divisao = spark.sql("select * from desafio_curso.tbl_divisao")

In [209]:
df_divisao = df_divisao.withColumn("division",col("division").cast(IntegerType()))

In [210]:
df_divisao.createOrReplaceTempView('tb_divisao')

# ENDEREÇO

In [260]:
df_endereco = spark.sql("select * from desafio_curso.tbl_endereco")

In [261]:
df_endereco.printSchema()

root
 |-- address_number: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- customer_address_1: string (nullable = true)
 |-- customer_address_2: string (nullable = true)
 |-- customer_address_3: string (nullable = true)
 |-- customer_address_4: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- dt_foto: string (nullable = true)



In [262]:
df_endereco = df_endereco.drop("customer_address_1","customer_address_2","customer_address_3","customer_address_4","zip_code")

In [263]:
#Convertendo os tipos de dados
df_endereco = df_endereco.withColumn("address_number",col("address_number").cast(IntegerType()))

In [264]:
#Tratando as colunas vazias
df_endereco = df_endereco.withColumn('city', regexp_replace('city', '^\s+$', 'Não Informado'))
df_endereco = df_endereco.select([when(col(c)=="",None).otherwise(col(c)).alias(c) for c in df_endereco.columns])
df_endereco = df_endereco.na.fill("Não Informado")

In [265]:
df_endereco.createOrReplaceTempView('tb_endereco')

In [266]:
df_endereco.show()

+--------------+----------------+-------+-------------+--------+
|address_number|            city|country|        state| dt_foto|
+--------------+----------------+-------+-------------+--------+
|      10000000|           Akron|     US|           OH|20230630|
|      10000453|   Não Informado|     UK|Não Informado|20230630|
|      10000455|Huntington Beach|     US|           CA|20230630|
|      10000456|        Edmonton|     CA|           AB|20230630|
|      10000458|         Saginaw|     US|           MI|20230630|
|      10000460|  Goodlettsville|     US|           TN|20230630|
|      10000461|    Boucherville|     CA|           QU|20230630|
|      10000462|       Hazelwood|     US|           MO|20230630|
|      10000466| North Highlands|     US|           CA|20230630|
|      10000469|        Montreal|     CA|           QU|20230630|
|      10000471|          Jeddah|     AU|Não Informado|20230630|
|      10000472|        Van Nuys|     US|           CA|20230630|
|      10000473|  Rancho 

# REGIÃO

In [267]:
df_regiao = spark.sql("select * from desafio_curso.tbl_regiao")

In [268]:
df_regiao.printSchema()

root
 |-- region_code: string (nullable = true)
 |-- region_name: string (nullable = true)
 |-- dt_foto: string (nullable = true)



In [269]:
#Convertendo os tipos de dados
df_regiao = df_regiao.withColumn("region_code",col("region_code").cast(IntegerType()))

In [270]:
df_regiao.createOrReplaceTempView('tb_regiao')

# VENDAS

In [271]:
df_vendas = spark.sql("select * from desafio_curso.tbl_vendas")

In [272]:
df_vendas.printSchema()

root
 |-- actual_delivery_date: string (nullable = true)
 |-- customerkey: string (nullable = true)
 |-- datekey: string (nullable = true)
 |-- discount_amount: string (nullable = true)
 |-- invoice_date: string (nullable = true)
 |-- invoice_number: string (nullable = true)
 |-- item_class: string (nullable = true)
 |-- item_number: string (nullable = true)
 |-- item: string (nullable = true)
 |-- line_number: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- order_number: string (nullable = true)
 |-- promised_delivery_date: string (nullable = true)
 |-- sales_amount: string (nullable = true)
 |-- sales_amount_based_on_list_price: string (nullable = true)
 |-- sales_cost_amount: string (nullable = true)
 |-- sales_margin_amount: string (nullable = true)
 |-- sales_price: string (nullable = true)
 |-- sales_quantity: string (nullable = true)
 |-- sales_rep: string (nullable = true)
 |-- u_m: string (nullable = true)
 |-- dt_foto: string (nullable = true)



In [223]:
#Convertendo os tipos de dados
df_vendas = df_vendas.withColumn("customerkey",col("customerkey").cast(IntegerType()))\
        .withColumn("discount_amount",regexp_replace("discount_amount", ',', '.').cast(DoubleType()))\
        .withColumn("invoice_number",col("invoice_number").cast(IntegerType()))\
        .withColumn("item_number",col("item_number").cast(IntegerType()))\
        .withColumn("line_number",col("item_number").cast(IntegerType()))\
        .withColumn("list_price",regexp_replace("list_price", ',', '.').cast(DoubleType()))\
        .withColumn("order_number",col("order_number").cast(IntegerType()))\
        .withColumn("sales_amount",regexp_replace("sales_amount", ',', '.').cast(DoubleType()))\
        .withColumn("sales_amount_based_on_list_price",regexp_replace("sales_amount_based_on_list_price", ',', '.').cast(DoubleType()))\
        .withColumn("sales_cost_amount",regexp_replace("sales_cost_amount", ',', '.').cast(DoubleType()))\
        .withColumn("sales_margin_amount",regexp_replace("sales_margin_amount", ',', '.').cast(DoubleType()))\
        .withColumn("sales_price",regexp_replace("sales_price", ',', '.').cast(DoubleType()))\
        .withColumn("sales_quantity",col("sales_quantity").cast(IntegerType()))\
        .withColumn("sales_rep",col("sales_rep").cast(IntegerType()))
df_vendas = df_vendas.select('discount_amount',
                             'invoice_number',
                             'item_class',
                             'item_number',
                             'item',
                             'line_number',
                             'list_price',
                             'order_number',
                             'sales_amount',
                             'sales_amount_based_on_list_price',
                             'sales_cost_amount',
                             'sales_margin_amount',
                             'sales_price',
                             'sales_quantity',
                             'sales_rep',
                             'u_m', 
                             'customerkey',
                             'dt_foto',
                             from_unixtime(unix_timestamp('actual_delivery_date', 'dd/MM/yyy')).alias('actual_delivery_date'),
                             from_unixtime(unix_timestamp('invoice_date', 'dd/MM/yyy')).alias('invoice_date'),
                             from_unixtime(unix_timestamp('promised_delivery_date', 'dd/MM/yyy')).alias('promised_delivery_date'),
                             from_unixtime(unix_timestamp('datekey', 'dd/MM/yyy')).alias('datekey')
                            )


In [224]:
#Tratando as colunas vazias
df_vendas = df_vendas.na.fill(value=0)
df_vendas = df_vendas.select([when(col(c)=="",None).otherwise(col(c)).alias(c) for c in df_vendas.columns])
df_vendas = df_vendas.na.fill("Não Informado")
df_vendas = df_vendas.withColumn("datekey",to_timestamp(col("datekey")))
df_vendas = df_vendas.withColumn("promised_delivery_date",to_timestamp(col("promised_delivery_date")))
df_vendas = df_vendas.withColumn("invoice_date",to_timestamp(col("invoice_date")))
df_vendas = df_vendas.withColumn("actual_delivery_date",to_timestamp(col("actual_delivery_date")))

In [225]:
df_vendas.printSchema()

root
 |-- discount_amount: double (nullable = true)
 |-- invoice_number: integer (nullable = true)
 |-- item_class: string (nullable = false)
 |-- item_number: integer (nullable = true)
 |-- item: string (nullable = false)
 |-- line_number: integer (nullable = true)
 |-- list_price: double (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- sales_amount: double (nullable = true)
 |-- sales_amount_based_on_list_price: double (nullable = true)
 |-- sales_cost_amount: double (nullable = true)
 |-- sales_margin_amount: double (nullable = true)
 |-- sales_price: double (nullable = true)
 |-- sales_quantity: integer (nullable = true)
 |-- sales_rep: integer (nullable = true)
 |-- u_m: string (nullable = false)
 |-- customerkey: integer (nullable = true)
 |-- dt_foto: string (nullable = false)
 |-- actual_delivery_date: timestamp (nullable = true)
 |-- invoice_date: timestamp (nullable = true)
 |-- promised_delivery_date: timestamp (nullable = true)
 |-- datekey: timestamp (nu

In [226]:
df_vendas.createOrReplaceTempView('tb_vendas')

# CRIANDO TABELÃO COM OS DADOS

In [236]:
#Criando tabelão com todos os dados
query='''
SELECT    v.*
FROM      tb_vendas v
          INNER JOIN tb_clientes c ON v.customerkey == c.customerkey
          INNER JOIN tb_regiao r ON c.region_code == r.region_code
          INNER JOIN tb_divisao d ON c.division == d.division
          LEFT JOIN tb_endereco e ON c.address_number == e.address_number
'''

In [237]:
df_stage = spark.sql(query)

In [238]:
df_stage.count()

65282

In [117]:
df_stage.show()

+-----------+-------------+-------------+---------------+-------------+--------+-------------+----------------+------------+-----------+-------------+------------------+-----------+-------------------+--------------------+---------------+-------------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+--------------+----+-------+------------------+------------------+------------------+------------------+-----+--------+-------+
|customerkey|     customer|customer_type|business_family|business_unit|division|division_name|line_of_business|       phone|region_code|  region_name|regional_sales_mgr|search_type|            datekey|actual_delivery_date|discount_amount|       invoice_date|invoice_number|item_class|item_number|                item|line_number|list_price|order_number|promised_delivery_

In [118]:
df_stage = (df_stage
            .withColumn('Ano', year(df_stage.invoice_date))
            .withColumn('Mes', month(df_stage.invoice_date))
            .withColumn('Dia', dayofmonth(df_stage.invoice_date))
            .withColumn('Trimestre', quarter(df_stage.invoice_date))
           )

df_stage = df_stage.select([when(col(c)=="",None).otherwise(col(c)).alias(c) for c in df_stage.columns])
df_stage = df_stage.na.fill("Não Informado")

In [119]:
df_stage.printSchema()

root
 |-- customerkey: integer (nullable = true)
 |-- customer: string (nullable = false)
 |-- customer_type: string (nullable = false)
 |-- business_family: string (nullable = false)
 |-- business_unit: integer (nullable = true)
 |-- division: integer (nullable = true)
 |-- division_name: string (nullable = false)
 |-- line_of_business: string (nullable = false)
 |-- phone: string (nullable = false)
 |-- region_code: integer (nullable = true)
 |-- region_name: string (nullable = false)
 |-- regional_sales_mgr: string (nullable = false)
 |-- search_type: string (nullable = false)
 |-- datekey: timestamp (nullable = true)
 |-- actual_delivery_date: timestamp (nullable = true)
 |-- discount_amount: double (nullable = true)
 |-- invoice_date: timestamp (nullable = true)
 |-- invoice_number: integer (nullable = true)
 |-- item_class: string (nullable = false)
 |-- item_number: integer (nullable = true)
 |-- item: string (nullable = false)
 |-- line_number: integer (nullable = true)
 |-- li

In [120]:
#Gerando keys para as DW

df_stage = df_stage.withColumn('key_cliente',sha2(col("customerkey").cast(StringType()),256))
df_stage = df_stage.withColumn('key_tempo',sha2(concat_ws('|', col('invoice_date'), col('Ano'),col('Mes'),col('Dia')),256))
df_stage = df_stage.withColumn('key_localidade',sha2(concat_ws('|', col('division'), col('region_code'),col('address_number')),256))

In [121]:
df_stage.createOrReplaceTempView('tb_stage')

In [122]:
df_stage.show()

+-----------+-------------+-------------+---------------+-------------+--------+-------------+----------------+------------+-----------+-------------+------------------+-----------+-------------------+--------------------+---------------+-------------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+--------------+----+-------+------------------+------------------+------------------+------------------+-----+--------+-------+----+---+---+---------+--------------------+--------------------+--------------------+
|customerkey|     customer|customer_type|business_family|business_unit|division|division_name|line_of_business|       phone|region_code|  region_name|regional_sales_mgr|search_type|            datekey|actual_delivery_date|discount_amount|       invoice_date|invoice_number|item_class|i

In [123]:
#DIM_CLIENTES
dim_clientes = spark.sql('''
    SELECT DISTINCT key_cliente
        ,business_family
        ,customer 
        ,customer_type 
        ,line_of_business
        ,regional_sales_mgr
        ,search_type
    FROM tb_stage    
''')

In [124]:
#Criando a dimensão de tempo
dim_tempo = spark.sql('''
    SELECT DISTINCT key_tempo
        ,invoice_date
        ,Ano 
        ,Mes 
        ,Dia
        ,Trimestre
    FROM tb_stage    
''')

dim_tempo = dim_tempo.withColumn('invoice_date',to_date('invoice_date'))

In [125]:
#Criando a dimensão de localidade
dim_localidade = spark.sql('''
    SELECT DISTINCT key_localidade
        ,division_name
        ,region_name 
        ,country 
        ,state
        ,city
        ,zip_code
    FROM tb_stage    
''')

In [129]:
#Criando a fato
ft_vendas = spark.sql('''
    SELECT DISTINCT key_cliente
        ,key_tempo
        ,key_localidade
        ,count(distinct invoice_number) qty_vendas
        ,sum(sales_quantity) quantity
        ,round(sum(sales_amount),2) amount
        ,round(sum(sales_cost_amount),2) cost
        ,round(sum(sales_amount - sales_cost_amount),2) total_amount
    FROM tb_stage    
    GROUP BY key_cliente
        ,key_tempo
        ,key_localidade
''')

In [131]:
def criar_csv (df,name):
    
    df.coalesce(1).write\
        .format('csv')\
        .option('header',True)\
        .mode('overwrite')\
        .option('sep',';')\
        .save("/datalake/gold/"+name)
    
    copiar = "hdfs dfs -get /datalake/gold/"+name+"/*.csv /input/desafio_curso/gold/"+name+".csv"
    
    os.system(copiar)
    
criar_csv(dim_tempo,'dim_tempo')
criar_csv(dim_localidade,'dim_localidade')
criar_csv(dim_clientes,'dim_clientes')
criar_csv(ft_vendas,'ft_vendas')
